In [1]:
from sage.rings.finite_rings.hom_finite_field import FiniteFieldHomomorphism_generic

# Goppa code

In [2]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite fiel `\GF{p^m}`
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are roots of `generating_pol
    """
    
    def __init__(self, defining_set, generating_pol):
        """
        Initialize.
        """
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = generating_pol.base_ring().prime_subfield()
        self._polynomial_ring = PolynomialRing(self._field, defining_set[0].parent().gen())
        
        if (not self._field.is_field() or not self._field.is_finite()):
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
    def get_generating_pol(self):
        return self._generating_pol
    
    def get_defining_set(self):
        return self._defining_set
    
    def get_parity_pol(self):
        parity_pol = list()
        
        for elem in self._defining_set:
            parity_pol.append((self._generating_pol.parent().gen() - elem).inverse_mod(self._generating_pol))
        
        return parity_pol
    
    def parity_check_matrix(self):
        """
        Return a parity check matrix of the code.
        """
        
        # create a matrix with the coefficients of the parity polynomial as columns
        h = self.get_parity_pol()
        H_aux = matrix(h[0].list())
        
        for i in range(1, len(h)):  
            aux = matrix(h[i].list())  # get coefficients of h[i]
            H_aux = H_aux.stack(aux)   # add at the end
            
        H_aux = H_aux.transpose()
        
        # calculate the maximum degree
        max_degree = 0
        
        for hi in H_aux:
            for hij in hi:
                if self._polynomial_ring(hij).degree() > max_degree:
                    max_degree = self._polynomial_ring(hij).degree()
        
        # extend each component to the base field
        H = Matrix(vector([0] * H_aux.ncols()))
 
        for hi in H_aux:
            matriz = Matrix(vector([0] * (max_degree + 1)).column())
            
            for hij in hi:
                aux = self._polynomial_ring(hij).list()
                
                while len(aux) != max_degree + 1:
                    aux.append(0)

                matriz = matriz.augment(vector(ZZ, aux).column())
            
            matriz = matriz.delete_columns([0])
            H = H.stack(matriz)
        
        H = H.delete_rows([0])
        
        return H
    
    def minimum_distance(self):
        """
        Return the minimum distance of the code.
        """
        return 2 * (self._generating_pol).degree() + 1
    
    def dimension(self):
        H = self.parity_check_matrix()
        
        return self._length - rank(H)
    
    def _repr_(self):
        """
        Representation of a Goppa code
        """
        return "[{}, {}] Goppa code".format(self._length, self.dimension())

# Goppa encoder

In [3]:
class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        H = self.code().parity_check_matrix()
        self._G = Matrix(self.code()._polynomial_ring, transpose(H).left_kernel().basis_matrix())
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self._G
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m``: a vector to encode
        """
        return m * self._G

    # TODO: add a method to add errors?
    
Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder

# Goppa decoder

In [4]:
# TODO
class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code``: code associated with the decoder
    """
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._defining_set = self.code().get_defining_set()
        
    def get_syndrome(self, c):
        """
        Return the syndrome polynomial
        """
        
        F = self.code()._field
        L = self._defining_set[0].parent()
        
        embFL = FiniteFieldHomomorphism_generic(Hom(F,L))
        secLF = embFL.section()
        
        h = self.code().get_parity_pol()
        
        syndrome = 0
        
        for i in range(len(h)):    
            syndrome = syndrome + embFL(c[i])*h[i]
            
        return syndrome
    
    def get_generating_pol(self):
        return self._generating_pol
    
    def decode_to_code(self, word):
        # TODO: no funciona
        i = 1
    
        # Paso 1
        S = self.get_syndrome(word)
        print("Sindrome: " + str(S))

        # Paso 2
        r_prev = self.get_generating_pol()
        t = self.get_generating_pol().degree()/2
        r_i = S
        U_prev = 0
        U_i = 1
        
        print("r_prev: " + str(r_prev))
        print("t: " + str(t))
        print("r_i: " + str(r_i))
        print("U_prev: " + str(U_prev))
        print("U_i: " + str(U_i))
        
        # Paso 3 y 4
        while r_i.degree() >= t:
            (d, u, v) = xgcd(r_prev, r_i)
            q_i = -v/u
            r_i = d/u
            U_i = q_i * U_i + U_prev
            r_prev = r_i
            U_prev = U_i
            i += 1

        # Paso 5
        eta = (-1)^i * r_i
        sigma = U_i

        return 0


Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

# Examples

In [5]:
F.<a> = GF(2^3)
a = F.gen()
R.<x> = F[]
g = x^2 + x + 1
defining_set = [i for i in F.list() if g(i) != 0]

In [6]:
# Goppa code
C = Goppa(defining_set, g)

print("Goppa code:")
print(C)

print("\nMinimun distance:")
print(C.minimum_distance())

print("\nParity polynomial:")
print(C.get_parity_pol())

print("\nParity matrix:")
print(C.parity_check_matrix())

Goppa code:
[8, 2] Goppa code

Minimun distance:
5

Parity polynomial:
[x + 1, a^2*x + a^2 + a + 1, (a^2 + a)*x + a + 1, a^2*x + a + 1, a*x + a^2 + 1, a*x + a^2 + a + 1, (a^2 + a)*x + a^2 + 1, x]

Parity matrix:
[1 1 1 1 1 1 1 0]
[0 1 1 1 0 1 0 0]
[0 1 0 0 1 1 1 0]
[1 0 0 0 0 0 0 1]
[0 0 1 0 1 1 1 0]
[0 1 1 1 0 0 1 0]


In [7]:
# Encoder of C
E = GoppaEncoder(C)

print("\nGoppa encoder:")
print(E)

print("\nGenerator matrix:")
print(E.get_generator_matrix())

word = vector(GF(2), (0, 1))

print("\nWord to encode:")
print(word)

y = E.encode(word)

print("\nWord encoded:")
print(y)

# add errors to y
y = y + vector(GF(2), (0, 0, 0, 0, 0, 1, 0, 0))

print("\nWord encoded with errors:")
print(y)
#print(y in C)


Goppa encoder:
Encoder for [8, 2] Goppa code

Generator matrix:
[1 0 0 1 0 1 1 1]
[0 1 1 1 1 1 1 0]

Word to encode:
(0, 1)

Word encoded:
(0, 1, 1, 1, 1, 1, 1, 0)

Word encoded with errors:
(0, 1, 1, 1, 1, 0, 1, 0)


In [8]:
# Decoder of C
D = GoppaDecoder(C)

print("\nDecode y\n:")

x = D.decode_to_code(y)

"""
print("\nWord decoded:")
print(x)
"""


Decode y
:
Sindrome: a*x + a^2 + a + 1
r_prev: x^2 + x + 1
t: 1
r_i: a*x + a^2 + a + 1
U_prev: 0
U_i: 1


AttributeError: 'sage.rings.fraction_field_element.FractionFieldElement_1poly_field' object has no attribute 'degree'

# **ALGORITMO DE SUGIYAMA**

1. Calcular el síndrome $S(x)$.
2. Sean $r_{-1}(x) = g(x)$, $r_0(x) = S(x)$, $U_{-1}(x) = 0$ y $U_0(x) = 1$.
3. Buscar $q_i(x)$ y $r_i(x)$ aplicando el algoritmo de Euclides para encontrar el máximo común divisor de $r_{i-2}(x)$ y $r_{i-1}(x)$ para $i = 0,..., k$, hasta que $k$ cumpla que $gr(r_{k-1}(x)) \geq t$ y $gr(r_k(x)) < t$:

$$r_{i-2}(x) = r_{i-1}(x) q_i(x) + r_i(x), \qquad gr(r_i(x)) < gr(r_{i-1})(x)$$
    
4. Calcular $U_k(x)$, donde
    
$$U_i(x) = q_i(x) U_{i-1}(x) + U_{i-2}(x)$$

5. La solución viene dada por:
$$\eta(x) = (-1)^k \delta r_k(x)$$
$$\sigma(x) = \delta U_k(x)$$